In [2]:
!pip install pandas numpy scikit-learn xgboost

     ------------------------------------- 150.0/150.0 MB 11.7 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Required Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier


In [4]:
# Load Dataset
file_path = "custom_industrial_dataset_127features.csv"  # Change path if needed
df = pd.read_csv(file_path)

# Features and Target
X = df.drop(columns=["Class", "SNR_Level"])
y = df["Class"]

# Encode Class Labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [5]:
# Define Models
models = {
    "SVM (RBF)": SVC(kernel="rbf", probability=True, random_state=42),
    "KNN (k=7)": KNeighborsClassifier(n_neighbors=7),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42),
    "Neural Network": MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42),
}

# Add Soft Voting Ensemble
ensemble = VotingClassifier(
    estimators=[
        ("rf", models["Random Forest"]),
        ("xgb", models["XGBoost"]),
        ("svm", models["SVM (RBF)"])
    ],
    voting="soft"
)
models["Ensemble"] = ensemble


In [6]:
# Evaluate Models
results = []
for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    results.append({
        "Model": name,
        "Accuracy": round(accuracy_score(y_test, y_pred), 4),
        "Precision": round(precision_score(y_test, y_pred, average='weighted'), 4),
        "Recall": round(recall_score(y_test, y_pred, average='weighted'), 4),
        "F1-Score": round(f1_score(y_test, y_pred, average='weighted'), 4)
    })


Training SVM (RBF)...
Training KNN (k=7)...
Training Random Forest...
Training XGBoost...


C:\Users\Yodhin Agarwal\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:46:40] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Training Neural Network...


C:\Users\Yodhin Agarwal\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Training Ensemble...


C:\Users\Yodhin Agarwal\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:47:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [7]:
# Display Results
results_df = pd.DataFrame(results).sort_values(by="F1-Score", ascending=False)
print("\nModel Performance Comparison:\n")
print(results_df.to_string(index=False))


Model Performance Comparison:

         Model  Accuracy  Precision  Recall  F1-Score
     SVM (RBF)     0.139     0.1341   0.139    0.1346
       XGBoost     0.128     0.1307   0.128    0.1273
     KNN (k=7)     0.122     0.1233   0.122    0.1193
 Random Forest     0.119     0.1224   0.119    0.1190
      Ensemble     0.119     0.1201   0.119    0.1178
Neural Network     0.106     0.1073   0.106    0.1066
